# First Practical Work

## Andres Navarro Pedregal
## NIA: 100451730

## Summary

In this practial work, we will be optimizing a portfolio of European ETF in order to maximize the projected return on investment given some fund contraints. This work will study different approaches for a fund allocation for different segments of the population like age groups, years to retire, goals, believes, etc. Moreover, we will make recommendation on the best approaches for them.

In order to perform the optimization, we will be using two methods:

1. The first method will consists on a continuous approach where we will try to maximize the percentage of ETFs to buy in order to get the maximum ROI. Moreover, we will add some constraints like the maximum risk, the maxium allocation in order to make a diversified portfolio and more.

2. And the second approach will add some binary varaibles in order to better fit the model to a real scenario such as only selecting a few funds or turning on or off different paramentes.

## Dataset

In order to study the different funds, we will be using a dataset consisting of all European ETFs given by MorningStar. It can be found here [link](https://www.kaggle.com/datasets/stefanoleone992/european-funds-dataset-from-morningstar). It is a dataset consisting of 9500 different funds with more than 132 entries (different types of performance analysis, returns, etc.)

In [24]:
import pandas as pd

data = pd.read_csv("ETFs.csv")
data

,ticker,isin,fund_name,inception_date,category,rating,analyst_rating,risk_rating,performance_rating,investment_strategy,...,fund_return_2016_q4,fund_return_2016_q3,fund_return_2016_q2,fund_return_2016_q1,fund_return_2015_q4,fund_return_2015_q3,fund_return_2015_q2,fund_return_2015_q1,quarters_up,quarters_down
0,0P00000C5S,CH0008899764,iShares SMI® ETF (CH),1999-10-06,Switzerland Equity,3.0,Negative,3.0,3.0,The Fund aims to achieve a return on your inve...,...,1.45,5.10,10.47,-4.09,4.18,-4.00,-3.96,10.56,15,8
1,0P00000FYY,SE0000693293,XACT OMXS30 UCITS ETF,2000-10-30,Sweden Equity,2.0,NaN,2.0,2.0,The fund is an exchange-traded index fund and ...,...,5.21,11.32,1.66,1.43,4.99,-6.15,-7.21,10.11,16,7
2,0P00000GCE,DE000A0H08N1,iShares STOXX Europe 600 Personal & Household ...,2002-07-08,Sector Equity Consumer Goods & Services,3.0,NaN,3.0,3.0,iShares STOXX Europe 600 Personal & Household ...,...,-1.51,5.34,8.17,6.48,4.83,4.12,-2.03,9.24,17,6
3,0P00000GF2,DE0006289465,iShares eb.rexx® Government Germany UCITS ETF ...,2003-02-04,EUR Government Bond,2.0,NaN,2.0,2.0,iShares eb.rexx Government Germany (DE) is an ...,...,-2.14,4.21,6.04,9.67,0.05,4.96,-3.72,-5.67,13,10
4,0P00000GKU,IE0032523478,iShares € Corp Bond Large Cap UCITS ETF EUR (D...,2003-03-17,EUR Corporate Bond,3.0,NaN,3.0,3.0,The investment objective of this Fund is to pr...,...,-2.74,5.58,6.26,10.55,1.38,3.78,-5.02,-5.84,16,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9490,0P0001K8E2,LU2099288503,Lyxor Core Global Government Bond (DR) UCITS E...,2020-06-24,Global Bond,NaN,NaN,NaN,NaN,The investment objective of MULTI UNITS LUXEMB...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
9491,0P0001K9DU,LU1220245556,Lyxor MSCI Pacific Ex Japan UCITS ETF - D-USD,2015-04-29,Pacific ex-Japan Equity,3.0,NaN,4.0,3.0,The Lyxor MSCI Pacific Ex Japan UCITS ETF is a...,...,2.25,11.22,8.06,4.26,11.13,-12.89,NaN,NaN,13,8
9492,0P0001K9DV,LU1923627332,Lyxor MSCI Russia UCITS ETF Dist,2019-03-14,Russia Equity,2.0,Negative,4.0,2.0,The investment objective of the MULTI UNITS LU...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,2
9493,0P0001K9DW,LU1650492256,MULTI-UNITS LUXEMBOURG - Lyxor FTSE 100 UCITS ...,2017-11-09,UK Large-Cap Equity,NaN,Bronze,NaN,NaN,The Lyxor FTSE 100 UCITS ETF is a UCITS compli...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,5


Here we can see the dataset which consists mainly on different ETFs which belongs to different sectors and have different characteristics. But there are many so let's only study a few of them so we can better use them to understand and get some insights.

We will only study the following characteristics:
- Fund name: name of the fund in order to know which fund we are working with.
- Sector_*: a number from 0 to 100 which tells the percentage of relation to that sector. We will be using this to make some correlation matrices and make our porfolio diversified.
- Management_Fees: this is the percentage of management fees the fund has per yearly basis. We want it as it influences the ROI at the end of the year.
- Sustainability_rank: this is a number from 0 to 5 given by MoringStar which tells how sustainable a fund is, being 5 the most sustainable.
- Fund_return_*: percentage return for a given year.
- Dividend_yield_factor: this it the percentage the fund has paid yearly on average with respect to its price.
- Assets_*: this is the percentage this fund has invested in the different assets classes (stocks, bonds, and cash). It will be used in order to adjust the conservativity of funds to be less volatile.
- Fund_size: this is the fund size in dollars. It will be used to select different strategies to make the allocation more towards larger or smaller funds.

In [25]:
columns = ["fund_name", 
"sector_basic_materials", 
"sector_consumer_cyclical", "sector_financial_services", "sector_real_estate", "sector_consumer_defensive", "sector_healthcare", "sector_utilities", "sector_communication_services", "sector_energy", "sector_industrials", "sector_technology",
"management_fees", "sustainability_rank",
"fund_return_2019", "fund_return_2018", "fund_return_2017", "fund_return_2016", "fund_return_2015" , 
"dividend_yield_factor" , "asset_stock" , "asset_bond" , "asset_cash" , "fund_size"]
df = data[columns]

#### Dataset Cleanup

First of all, we have to check if our dataset is properly formatted and if it has null values. So let's start checking it.

In [26]:
df.isna().sum()

fund_name                           0
sector_basic_materials           4576
sector_consumer_cyclical         4385
sector_financial_services        4510
sector_real_estate               5081
sector_consumer_defensive        4685
sector_healthcare                4838
sector_utilities                 5008
sector_communication_services    4490
sector_energy                    5029
sector_industrials               4403
sector_technology                4705
management_fees                   409
sustainability_rank              3096
fund_return_2019                 1834
fund_return_2018                 3161
fund_return_2017                 3865
fund_return_2016                 4606
fund_return_2015                 5202
dividend_yield_factor            3502
asset_stock                         0
asset_bond                          0
asset_cash                          0
fund_size                          14
dtype: int64

Here we can see the different entries by columns that have null values. So let's first fix them so we don't have missing data when we are working with it.

First of all, we will be removing all funds that do not have a fund size or management_fees as they are less than 500 entries and they are not that significant.

In [27]:
df = df.dropna(subset=["fund_size"])
df = df.dropna(subset=["management_fees"])


Now, we will properly format the percentage variables so they are from 0 to 1. This way the formulas will be much easier.

In [28]:
df["management_fees"] = df["management_fees"] / 100
df["dividend_yield_factor"] = df["dividend_yield_factor"] / 100

asset_columns = df.filter(regex="asset_*").columns
df[asset_columns] = df[asset_columns] / 100

print(df.isna().sum())

fund_name                           0
sector_basic_materials           4309
sector_consumer_cyclical         4127
sector_financial_services        4239
sector_real_estate               4801
sector_consumer_defensive        4416
sector_healthcare                4568
sector_utilities                 4732
sector_communication_services    4232
sector_energy                    4752
sector_industrials               4137
sector_technology                4440
management_fees                     0
sustainability_rank              2880
fund_return_2019                 1640
fund_return_2018                 2875
fund_return_2017                 3569
fund_return_2016                 4292
fund_return_2015                 4876
dividend_yield_factor            3242
asset_stock                         0
asset_bond                          0
asset_cash                          0
fund_size                           0
dtype: int64


Now, we will take care of the dividend_yield_factor and the sutainability_rank. This two we will make the empty values 0 as this will assume the worst case scenario, but we don't have to delete them.

In [29]:
df["dividend_yield_factor"] = df["dividend_yield_factor"].fillna(0)
df["sustainability_rank"] = df["sustainability_rank"].fillna(0)

And for the sectors we will do something similar, make them 0. This will make the funds that are empty for a specific sector assume that they do not have anything with that sector.

In [30]:
sector_columns = df.filter(regex="sector_*").columns
df[sector_columns] = df[sector_columns].fillna(0)

Now let's check the other empty values we have left regarding the fund return.

In [31]:

print(df.isna().sum())
df.shape

fund_name                           0
sector_basic_materials              0
sector_consumer_cyclical            0
sector_financial_services           0
sector_real_estate                  0
sector_consumer_defensive           0
sector_healthcare                   0
sector_utilities                    0
sector_communication_services       0
sector_energy                       0
sector_industrials                  0
sector_technology                   0
management_fees                     0
sustainability_rank                 0
fund_return_2019                 1640
fund_return_2018                 2875
fund_return_2017                 3569
fund_return_2016                 4292
fund_return_2015                 4876
dividend_yield_factor               0
asset_stock                         0
asset_bond                          0
asset_cash                          0
fund_size                           0
dtype: int64


(9081, 24)

Here we see that there are quite a few funds with empty return. We will make them 0 to assume they did not return anything. Ans also we will scale them down to be from 0 to 1 as explained before.

In [32]:
fund_columns = df.filter(regex="fund_return").columns
df[fund_columns] = df[fund_columns].fillna(0) / 100
print(df.isna().sum())

fund_name                        0
sector_basic_materials           0
sector_consumer_cyclical         0
sector_financial_services        0
sector_real_estate               0
sector_consumer_defensive        0
sector_healthcare                0
sector_utilities                 0
sector_communication_services    0
sector_energy                    0
sector_industrials               0
sector_technology                0
management_fees                  0
sustainability_rank              0
fund_return_2019                 0
fund_return_2018                 0
fund_return_2017                 0
fund_return_2016                 0
fund_return_2015                 0
dividend_yield_factor            0
asset_stock                      0
asset_bond                       0
asset_cash                       0
fund_size                        0
dtype: int64


### Feautre Engineering

Now that we have all our dataset cleaned and properly formatted, we can now do some feature engineering to get other variables that will be more useful in our study. First of all we will be creating two new variables:

1. Fund return: this will be the mean of the return of the funds from 2015 to 2020. We will be using this to calculate the fund return on average as it is the best predictor for the future.

2. Fund risk: this will be the standard deviation of the years from 2015 to 2020. This will be used to assess the volatility of the fund so we can make our porfolio more or less volatile therefore riskier.

In [33]:
df["fund_return"] = df.filter(regex="fund_return*").mean(axis=1)
df["fund_risk"] = df.filter(regex="fund_return*").std(axis=1)

In [36]:
test = df.loc[1:10]
corr_matrix = test[sector_columns].T.corr()
corr_matrix = corr_matrix.fillna(0)

In [37]:
model.pprint()

1 Set Declarations
    Funds : Funds
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

9 Param Declarations
    AssetBondPercentage : Asset Bond Percentage
        Size=10, Index=Funds, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :    0.0
          2 :    0.0
          3 : 0.9986
          4 : 0.9604
          5 : 0.9647
          6 :    0.0
          7 :    0.0
          8 :    0.0
          9 :    0.0
         10 :    0.0
    AssetCashPercentage : Asset Cash Percentage
        Size=10, Index=Funds, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :                0.0001
          2 :                0.0054
          3 : 0.0014000000000000002
          4 :               -0.0003
          5 :                0.0042
          6 :  0.006999999999999999
          7 :                0.0037
          8 :                   0.0
   

# EXTRA

 

In [38]:

And now we will get the correlation matrix between the sector variables of the funds. This will tell us how related two funds are to each other and will allow us to make our portfolio more diversified so we don't end up with funds that are all from the same sector as this will be bad in the future if something bad happens to that sector.

SyntaxError: unterminated string literal (detected at line 1) (3619037012.py, line 1)

In [ ]:
#corr_matrix = df[sector_columns].T.corr()
#corr_matrix = corr_matrix.fillna(0)
#print(corr_matrix)

In [ ]:
# corr_matrix_pyomo = {}
# Iterate over the rows and columns of the correlation matrix
# for fund_i_index, fund_i in enumerate(corr_matrix.index):
#     print(fund_i_index)
#     for fund_j_index, fund_j in enumerate(corr_matrix.index):
#         print(fund_j)
#         print(corr_matrix)
#         corr_matrix_pyomo[(fund_i_index, fund_j_index)] = corr_matrix[fund_i_index, fund_j_index]
#         
# corr_matrix_pyomo"